Including code to insert data into MySQL DataBase

In [2]:
%pip install schedule


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import time
import mysql.connector
from datetime import datetime
import re
import schedule
import time

In [4]:
# MySQL configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'RootPass@#007',
    'database': 'price_tracker_db'
}

# Function to connect to MySQL and insert data collected from websites
def insert_data_into_db(product_name, price, source_url, website):
    try:
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()
        product_id_mapping = {
            'AirPods Max': 1,
            'Ring Video Doorbell': 2,
            'Instant Pot Pro': 3,
            'Dyson V8 Absolute': 5,
            'Meta Quest 3': 4,
        }

        # Insert data into the 'products' table
        insert_query = "INSERT INTO products (product_name, price, source_url, website, scrape_date, p_id) VALUES (%s, %s, %s, %s, %s, %s)"
        data = (product_name, price, source_url, website, datetime.now(), product_id_mapping[product_name])
        cursor.execute(insert_query, data)

        conn.commit()
        print(f"Data inserted into database for {product_name} successfully.")
    except Exception as e:
        print(f"Error inserting data into database: {str(e)}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

# Function to scrape data from Amazon
def scrape_amazon(url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(5)

    page_content = driver.page_source
    driver.quit()

    soup = BeautifulSoup(page_content, 'html.parser')

    product_title_span = soup.find('span', {'id': 'productTitle', 'class': 'a-size-large'})
    product_title = product_title_span.get_text(strip=True)

    price_span = soup.find('div', {'id': 'corePrice_feature_div'}).find('span', {'class': 'a-offscreen'})
    price = price_span.get_text(strip=True)

  # Convert the price to float, assuming it's in the format "$99.99"
    price = float(price.replace('$', ''))
    print("Amazon - Product Name:", product_title)
    print("Amazon - Price:", price)
    print("")
    return price

# Function to scrape data from Walmart

def scrape_walmart(url):
    headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}

    with requests.Session() as session:
        response = session.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            name_element = soup.find("h1", {"id": "main-title", "itemprop": "name"})
            product_name = name_element.text.strip() if name_element else "Name not found"

            # Extract the price text, including additional characters
            price_element = soup.find("span", {"itemprop": "price", "aria-hidden": "false"})
            price_text = price_element.text.strip() if price_element else "Price not found"

            # Use a regular expression to extract the numeric part from the price text
            match = re.search(r'\d+\.\d+', price_text)
            product_price = float(match.group()) if match else None

            print("Walmart - Product Name:", product_name)
            print("Walmart - Price:", product_price)
            print("")
        else:
            print("Failed to retrieve Walmart page. Status code:", response.status_code)
            print("")

    return product_price


# Function to scrape data from Best Buy
def scrape_best_buy(url):
    headers = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}

    with requests.Session() as session:
        response = session.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            name_element = soup.find("div", {"class": "sku-title"}).find("h1", {"class": "heading-5"})
            product_name = name_element.text.strip() if name_element else "Name not found"

            price_element = soup.find("div", {"class": "priceView-hero-price"}).find("span", {"aria-hidden": "true"})
            product_price = price_element.text.strip() if price_element else "Price not found"

            # Convert the price to float, assuming it's in the format "$99.99"
            product_price = float(product_price.replace('$', ''))

            print("Best Buy - Product Name:", product_name)
            print("Best Buy - Price:", product_price)
            print("")
        else:
            print("Failed to retrieve Best Buy page. Status code:", response.status_code)
            print("")

    return product_price

# List of product URLs
product_urls = {
    'AirPods Max': [
        'https://www.amazon.com/New-Apple-AirPods-Max-Space/dp/B08PZJN7BD?th=1',
        'https://www.walmart.com/ip/Apple-AirPods-Max-Space-Gray/509451094',
        'https://www.bestbuy.com/site/apple-airpods-max-space-gray/6373460.p?skuId=6373460'
    ],

    'Ring Video Doorbell': [
        'https://www.amazon.com/Ring-Video-Doorbell-Satin-Nickel-2020-Release/dp/B08N5NQ869/ref=sr_1_1?keywords=ring+doorbell&qid=1699644293&sr=8-1',
        'https://www.walmart.com/ip/Ring-Video-Doorbell-Battery-or-Wired-Bronze-1080p-HD-Free-Cleaning-Cloth-Wi-fi-Home-Security-5-in/5097803945?from=/search',
        'https://www.bestbuy.com/site/ring-video-doorbell-venetian-bronze/6441423.p?skuId=6441423'
    ],

    'Instant Pot Pro': [
        'https://www.amazon.com/Instant-Pot-Pressure-Steamer-Sterilizer/dp/B08PQ2KWHS',
        'https://www.walmart.com/ip/Instant-Pot-Pro-6-Quart-Electric-Pressure-Cooker-10-in-1-Slow-Rice-Grain-Steamer-Saut-Sous-Vide-Yogurt-Maker-Sterilizer-Warmer/674041054?from=/search',
        'https://www.bestbuy.com/site/instant-pot-6qt-pro-electric-pressure-cooker-black/6452130.p?skuId=6452130'
    ],

    'Dyson V8 Absolute': [
        'https://www.amazon.com/Dyson-V8-Absolute-Cordless-Vacuum/dp/B01EX9D3UW',
        'https://www.walmart.com/ip/Dyson-V8-Absolute-Cordless-Vacuum-First-Generation-Silver-New/1068750176',
        'https://www.bestbuy.com/site/dyson-v8-cordless-vacuum-silver-nickel/6500879.p?skuId=6500879'
    ],

    'Meta Quest 3': [
        'https://www.amazon.com/Meta-Quest-128GB-Breakthrough-Reality-3/dp/B0C8VKH1ZH',
        'https://www.walmart.com/ip/Meta-Quest-3-128GB-Breakthrough-Mixed-Reality-Powerful-Performance-Asgard-s-Wrath-2-Bundle/2212742363?adsRedirect=true',
        'https://www.bestbuy.com/site/meta-quest-3-breakthrough-mixed-reality-128gb-white/6549064.p?skuId=6549064'
    ],
}

# Scrape data for each product
def scraping():
    for product_name, urls in product_urls.items():
        print(f"Scraping data for {product_name}")
        for url in urls:
            if 'amazon' in url:
                price = scrape_amazon(url)
                insert_data_into_db(product_name, price, url, 'amazon')
            elif 'walmart' in url:
                price = scrape_walmart(url)
                insert_data_into_db(product_name, price, url, 'walmart')
            elif 'bestbuy' in url:
                price = scrape_best_buy(url)
                insert_data_into_db(product_name, price, url, 'bestbuy')
    
if __name__ == "__main__":
    # Every day at 12am or 00:00 time bedtime() is called.
    schedule.every().day.at("12:26").do(scraping)
    while True:
        # Checks whether a scheduled task 
        # is pending to run or not
        schedule.run_pending()
        time.sleep(10)

# scraping()

Scraping data for AirPods Max
Amazon - Product Name: Apple AirPods Max Wireless Over-Ear Headphones, Active Noise Cancelling, Transparency Mode, Personalized Spatial Audio, Dolby Atmos, Bluetooth Headphones for iPhone – Sky Blue
Amazon - Price: 529.0

Data inserted into database for AirPods Max successfully.
Walmart - Product Name: Apple AirPods Max - Space Gray
Walmart - Price: 629.0

Data inserted into database for AirPods Max successfully.
Best Buy - Product Name: Apple - AirPods Max - Space Gray
Best Buy - Price: 479.99

Data inserted into database for AirPods Max successfully.
Scraping data for Ring Video Doorbell
Amazon - Product Name: Ring Video Doorbell - 1080p HD video, live notifications when away from home, simple setup, privacy controls | Satin Nickel
Amazon - Price: 54.99

Data inserted into database for Ring Video Doorbell successfully.
Walmart - Product Name: Ring_Video Doorbell Battery or Wired, Bronze, 1080p HD, Free Cleaning Cloth, Wi-fi Home Security, 5 in
Walmart - 